In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
path = 'https://raw.githubusercontent.com/AlexeyKamaev/test/main/_data.csv'
df = pd.read_csv(path)
pd.set_option('display.max_columns', None)

In [3]:
df = df.rename(columns={'Площадь, м2':'Площадь'}).drop('Ссылка на объявление',axis = 1).drop('Unnamed: 0',axis = 1).drop('Телефоны',axis = 1)
df = df[df['Адрес'].str.contains('Москва')]
df = df.dropna(subset=['Метро'])

In [4]:
def get_price(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  return int(r.split()[0])

def get_deposit(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  try:
      if int(r.split()[2]) < 1000 or int(r.split()[2]) > int(r.split()[0]) * 6:
        return int(r.split()[0])
      else:
        return int(r.split()[2])
  except:
      return int(r.split()[0])
    
x1 = df['Цена'].map(get_price) 
x2 = df['Цена'].map(get_deposit) 

In [5]:
df.insert(9, "Цена за месяц",x1)
df.insert(10,'Залог',x2)

In [6]:
def first_element(x):
    y = re.findall(r'\d+', x)
    return y[0]

df['Этаж'] = df['Дом'].map(first_element).astype('int')
def second_element(x):
    y = re.findall(r'\d+', x)
    return y[1]

df['Этажность дома'] = df['Дом'].apply(second_element).astype('int')
from operator import methodcaller
def func(x):
  return float(x[0])
point_place = df['Площадь'].map(methodcaller("split", "/")).map(func).copy()
df.insert(5, "Общая площадь",point_place)

In [7]:
def func (y):
    y[:y.find('(')]
    return y[:y.find('(')].strip()

def func2 (y):
    y[:y.find('(')]
    return y[y.find('м. ')+3:].strip()

x = df['Метро'].map(func).map(func2).copy()
df.insert(3, "Станция", x)

In [8]:
#чистка
df.drop(['Метро', 'Цена','Описание','Дополнительно','Название ЖК'], axis= 1, inplace=True)
df = df[['ID  объявления','Адрес','Станция','Серия дома', 'Общая площадь',	'Площадь', 'Этаж',	'Этажность дома', 'Высота потолков, м', 'Площадь комнат, м2', 'Количество комнат','Ремонт','Балкон', 'Окна','Санузел','Лифт','Мусоропровод', 'Тип','Дом',	'Парковка',	'Цена за месяц','Залог','Можно с детьми/животными']].copy()
df = df.rename(columns={'ID  объявления':'ID'}).reset_index(drop=True)

In [9]:
df

,ID,Адрес,Станция,Серия дома,Общая площадь,Площадь,Этаж,Этажность дома,"Высота потолков, м","Площадь комнат, м2",Количество комнат,Ремонт,Балкон,Окна,Санузел,Лифт,Мусоропровод,Тип,Дом,Парковка,Цена за месяц,Залог,Можно с детьми/животными
0,271271157,"Москва, улица Новый Арбат, 27",Смоленская,NaN,200.0,200.0/20.0,5,16,3.00,NaN,4,Дизайнерский,NaN,NaN,NaN,"Пасс (4), Груз (1)",Да,Квартира,"5/16, Монолитный",подземная,500000,500000,"Можно с детьми, Можно с животными"
1,271634126,"Москва, улица Новый Арбат, 27",Смоленская,NaN,198.0,198.0/95.0/18.0,5,16,3.50,25 25 20 25,4,Дизайнерский,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)","Пасс (1), Груз (1)",Нет,Квартира,"5/16, Монолитно-кирпичный",подземная,500000,500000,Можно с детьми
2,271173086,"Москва, улица Новый Арбат, 27",Смоленская,NaN,200.0,200.0/116.0/4.0,5,16,3.20,NaN,"4, Оба варианта",Евроремонт,NaN,На улицу и двор,Совмещенный (3),Пасс (1),NaN,Квартира,5/16,подземная,500000,500000,Можно с детьми
3,272197456,"Москва, переулок Плотников, 21С1",Смоленская,NaN,170.0,170.0/95.0/17.0,5,6,3.20,14-42-20-19,"4, Оба варианта",Евроремонт,NaN,На улицу и двор,Совмещенный (3),Пасс (1),NaN,Квартира,5/6,подземная,400000,400000,Можно с животными
4,273614615,"Москва, улица Новый Арбат, 15",Арбатская,NaN,58.0,58.0/38.0/5.0,12,26,3.90,20 18,2,Евроремонт,NaN,На улицу и двор,Совмещенный (2),"Пасс (1), Груз (1)",Да,Квартира,"12/26, Панельный",NaN,225000,225000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19386,215565511,"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",Говорово,NaN,35.0,35.0/16.4/8.0,10,14,NaN,NaN,NaN,Евроремонт,Балкон (1),NaN,Совмещенный (1),"Пасс (1), Груз (1)",NaN,Квартира,10/14,NaN,42000,42000,NaN
19387,274654844,"Москва, Производственная улица, 8к1, ш. Боровс...",Солнцево,NaN,38.7,38.7/16.5/11.0,5,18,NaN,NaN,1,Евроремонт,Лоджия (1),Во двор,Совмещенный (1),"Пасс (1), Груз (1)",NaN,Квартира,"5/18, Монолитный",NaN,45000,45000,NaN
19388,268679909,"Москва, Боровский проезд, 11",Солнцево,NaN,43.1,43.1,5,5,NaN,NaN,"2, Оба варианта",Дизайнерский,Балкон (1),На улицу и двор,Совмещенный (1),NaN,NaN,Квартира,"5/5, Кирпичный",NaN,50000,50000,Можно с детьми
19389,274807525,"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",Солнцево,NaN,52.5,52.5/10.0,8,23,2.65,NaN,2,Евроремонт,Лоджия (1),Во двор,"Совмещенный (1), Раздельный (1)",Пасс (3),Да,Квартира,"8/23, Монолитный",наземная,55000,50000,NaN


In [10]:
metro = pd.read_csv('metro - metro.csv', sep=',').rename(columns={'Station':'Станция'})
def func(s):
    s = s.replace('ё','е')
    s = s.replace('  ',' ')
    if s == 'Марк':
        s = 'Алтуфьево'
    elif s == 'Перерва':
        s = 'Братиславская'
    elif s.lower() == 'библиотека им. ленина':
        s = 'библиотека имени ленина'
    elif s.lower() == 'бескудниково':
        s = 'селигерская'
    elif s.lower() == 'гражданская':
        s = 'дмитровская'
    return s.lower().strip()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19391 entries, 0 to 19390
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        19391 non-null  int64  
 1   Адрес                     19391 non-null  object 
 2   Станция                   19391 non-null  object 
 3   Серия дома                2088 non-null   object 
 4   Общая площадь             19391 non-null  float64
 5   Площадь                   19391 non-null  object 
 6   Этаж                      19391 non-null  int64  
 7   Этажность дома            19391 non-null  int64  
 8   Высота потолков, м        10369 non-null  float64
 9   Площадь комнат, м2        12306 non-null  object 
 10  Количество комнат         18868 non-null  object 
 11  Ремонт                    16944 non-null  object 
 12  Балкон                    12853 non-null  object 
 13  Окна                      14333 non-null  object 
 14  Санузе

In [20]:
#координаты метро

metro = metro.drop_duplicates('Станция').loc[:,['Станция','coor']]
metro.loc[:,'Станция'] = metro.loc[:,'Станция'].map(func)
df.loc[:,'Станция'] = df.loc[:,'Станция'].map(func)

m = df.merge(metro, how = 'left', on='Станция').dropna(how='all')
df.insert(3, "metro_coord",m['coor'])

In [22]:
df.head(3)

,ID,Адрес,Станция,metro_coord,Серия дома,Общая площадь,Площадь,Этаж,Этажность дома,"Высота потолков, м","Площадь комнат, м2",Количество комнат,Ремонт,Балкон,Окна,Санузел,Лифт,Мусоропровод,Тип,Дом,Парковка,Цена за месяц,Залог,Можно с детьми/животными
0,271271157,"Москва, улица Новый Арбат, 27",смоленская,"(55.7488802, 37.5830392)",NaN,200.0,200.0/20.0,5,16,3.0,NaN,4,Дизайнерский,NaN,NaN,NaN,"Пасс (4), Груз (1)",Да,Квартира,"5/16, Монолитный",подземная,500000,500000,"Можно с детьми, Можно с животными"
1,271634126,"Москва, улица Новый Арбат, 27",смоленская,"(55.7488802, 37.5830392)",NaN,198.0,198.0/95.0/18.0,5,16,3.5,25 25 20 25,4,Дизайнерский,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)","Пасс (1), Груз (1)",Нет,Квартира,"5/16, Монолитно-кирпичный",подземная,500000,500000,Можно с детьми
2,271173086,"Москва, улица Новый Арбат, 27",смоленская,"(55.7488802, 37.5830392)",NaN,200.0,200.0/116.0/4.0,5,16,3.2,NaN,"4, Оба варианта",Евроремонт,NaN,На улицу и двор,Совмещенный (3),Пасс (1),NaN,Квартира,5/16,подземная,500000,500000,Можно с детьми


In [50]:
#Координаты адреса дома
house = pd.read_csv('coordinates.csv', sep=',')
house = house.rename(columns={'ID  объявления':'ID'}).reset_index(drop=True).drop('Unnamed: 0', axis=1)
me = df.merge(house, how = 'left', on='ID').dropna(how='all')

In [48]:
df.insert(2, "house_coord",me['coor'])

In [52]:
df.insert(5,'house_metro',0)
df.insert(6,'house_kremlin',0)

df.head()

,ID,Адрес,house_coord,Станция,metro_coord,house_metro,house_kremlin,Серия дома,Общая площадь,Площадь,Этаж,Этажность дома,"Высота потолков, м","Площадь комнат, м2",Количество комнат,Ремонт,Балкон,Окна,Санузел,Лифт,Мусоропровод,Тип,Дом,Парковка,Цена за месяц,Залог,Можно с детьми/животными
0,271271157,"Москва, улица Новый Арбат, 27","(55.75184675, 37.57974750955721)",смоленская,"(55.7488802, 37.5830392)",0,0,NaN,200.0,200.0/20.0,5,16,3.0,NaN,4,Дизайнерский,NaN,NaN,NaN,"Пасс (4), Груз (1)",Да,Квартира,"5/16, Монолитный",подземная,500000,500000,"Можно с детьми, Можно с животными"
1,271634126,"Москва, улица Новый Арбат, 27","(55.75184675, 37.57974750955721)",смоленская,"(55.7488802, 37.5830392)",0,0,NaN,198.0,198.0/95.0/18.0,5,16,3.5,25 25 20 25,4,Дизайнерский,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)","Пасс (1), Груз (1)",Нет,Квартира,"5/16, Монолитно-кирпичный",подземная,500000,500000,Можно с детьми
2,271173086,"Москва, улица Новый Арбат, 27","(55.75184675, 37.57974750955721)",смоленская,"(55.7488802, 37.5830392)",0,0,NaN,200.0,200.0/116.0/4.0,5,16,3.2,NaN,"4, Оба варианта",Евроремонт,NaN,На улицу и двор,Совмещенный (3),Пасс (1),NaN,Квартира,5/16,подземная,500000,500000,Можно с детьми
3,272197456,"Москва, переулок Плотников, 21С1",NaN,смоленская,"(55.7488802, 37.5830392)",0,0,NaN,170.0,170.0/95.0/17.0,5,6,3.2,14-42-20-19,"4, Оба варианта",Евроремонт,NaN,На улицу и двор,Совмещенный (3),Пасс (1),NaN,Квартира,5/6,подземная,400000,400000,Можно с животными
4,273614615,"Москва, улица Новый Арбат, 15","(55.7521355, 37.5853395)",арбатская,"(55.7522109, 37.6087395)",0,0,NaN,58.0,58.0/38.0/5.0,12,26,3.9,20 18,2,Евроремонт,NaN,На улицу и двор,Совмещенный (2),"Пасс (1), Груз (1)",Да,Квартира,"12/26, Панельный",NaN,225000,225000,NaN


# Расстояния
1. Расстояние до Кремля от дома: house_kremlin
2. Расстояние от дома до метро: house_metro
3. Если нет координаты дома, то среднее расстоние до метро в районе // ср расстония от района до кремля
4. Если нет координат метро, то макс расстояние до метро + макс расстояние до креля

In [53]:
df.to_csv('data.csv')